In [1]:
import main

In [2]:
words = main.load_txt_file()

In [2]:
client = main.instantiate_weaviate()

Skipping class creation


In [4]:
client.schema.delete_all()

In [6]:
from pathlib import Path
filepath = Path("./data/kubernetes_concepts_overview.txt")
src_data = main.SourceData(
    source_path=filepath,
    source_text=main.load_data(filepath)
)

In [7]:
main.add_to_weaviate(src_data, client)

13

In [8]:
main.generate_summary_with_weaviate("What topics does this cover?", client)

'What topics does this cover?\n\nThis text covers an overview of Kubernetes, its features, and benefits. It also discusses the evolution of deployment and containerization, as well as what Kubernetes is not.'

In [33]:
source_topic = "database"

topic_prompt = f"""
If the following text does includes information about {source_topic}, extract a list of three to six related sub-topics
related to {source_topic} that the user might learn about.
Deliver the topics as a short list, each separated by two consecutive newlines like `\n\n`

If the following information does not includes information about {source_topic}, tell the user that the information could not be found.
=====
"""

response = (
    client.query.get(main.WV_CLASS, ["body"])
    .with_near_text({"concepts": [source_topic]})
    .with_limit(3)
    .with_generate(
        grouped_task=topic_prompt
    )
    .do()
)

main.get_generated_result(response)

'- Data modeling\n- Query languages\n- Security and privacy\n- Distributed computing\n- Relational databases\n- Non-relational databases (NoSQL)'

In [30]:
for wiki_title in [
    "Database",
    "Vector database",
    "Containerization (computing)",
    "Graphite",
]:
    src_data = main.SourceData(
        source_path=wiki_title,
        source_text=main.load_wiki_page(wiki_title)
    )
    main.add_to_weaviate(src_data, client)

In [12]:
main.generate_summary_with_weaviate("graphite", client)

'Graphite is a crystalline form of the element carbon and is consumed on a large scale for uses in pencils, lubricants, and electrodes.'

In [5]:
main.generate_summary_with_weaviate("nosql database", client)

'NoSQL databases are a type of non-relational database that use different query languages than traditional relational databases, which use SQL for writing and querying data.'

In [7]:
main.suggest_topics_with_weaviate("database", client)

'- Data modeling\n- Query languages\n- Security and privacy of sensitive data'

In [8]:
main.suggest_topics_with_weaviate("computers", client)

'- Database models and design\n- Database management systems (DBMS)\n- Relational and non-relational databases\n- Virtualization and containerization\n- Kubernetes and distributed systems\n- Vector databases and embeddings'